## Project: **Finding Lane Lines on the Road** 

**Pipeline**

Pipeline for this project consist of the following steps:
- Converting image to gray scale after color selection to help capture lane lines
- Use Canny edge detection to detect edges in the images
- Use Hough Transform to convert detected edges to lines in the region of interest in the image
- Use least square fit to find the linear line which are the lane lines in the images 
- Combine the lane lines found with the original image and output final image
- Process video clips as images and apply the above steps to add lane lines to output clip


Packages used for image processing

In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.pyplot import cm
import numpy as np
import cv2
%matplotlib inline

Packages and function used for os to create folders

In [2]:
import os
os.listdir("test_images/")

def create_dir(dir_name):
    try:
        os.stat(dir_name)
    except:
        os.mkdir(dir_name)

**Image Processing**

Below are the two main types of lane lines detected in the project, one type is white lane lines, the other is yellow lane lines. 

<figure>
 <img src="test_images/solidWhiteCurve.jpg" width="320" alt="Combined Image" />
 <img src="test_images/solidYellowCurve.jpg" width="320" alt="Combined Image" />
</figure>

After some iterations based on different images with different brightness and shading (especially after the challenge.mp4 video), the approach chosen was to first convert the image to HLS color scale and filter only white and yellow from the images. Below plots show the result after converting the images to gray scale, which we can clearly detect the lane lines. 

<figure>
 <img src="test_images_gray/solidWhiteCurve.jpg" width="320" alt="Combined Image" />
 <img src="test_images_gray/solidYellowCurve.jpg" width="320" alt="Combined Image" />
</figure>

Next step was to apply Canny edge detection (by using cv2.Canny) to capture edges. Different Gaussian filter kernel size was tested to smooth out the unwanted objects in the plot. Final kernel size used in the project was 13. 

Below plot shows the region of interest in the images used since the rest of the images will not contain the lane lines.

<figure>
 <img src="examples/region1.jpg" width="320" alt="Combined Image" />
</figure>

After Hough transform (by using cv2.HoughLinesP), lane lines were found as shown in plots below. 

<figure>
 <img src="test_images_output_no_averaging/solidWhiteCurve.jpg" width="320" alt="Combined Image" />
 <img src="test_images_output_no_averaging/solidYellowCurve.jpg" width="320" alt="Combined Image" />
</figure>

Next step was to take the lines from Hough transform and use least square fit to find the optimal line to represent the lane lines. The idea was to get the left and right lane lines by distinghish the lane lines based on positive (right) and negative (left) slopes. However, as discovered while checking the result from challenge.mp4 video, there could be 
lines with different slopes on the opposite of the plot which will cause some jumpy lines in the video. To resolve this issue, a boundary condition was set to ensure that the right and left lane lines stay on proper side of the image.

<figure>
 <img src="test_images_output/solidWhiteCurve.jpg" width="320" alt="Combined Image" />
 <img src="test_images_output/solidYellowCurve.jpg" width="320" alt="Combined Image" />
</figure>

Functions used for image processing

In [3]:
import math

def rgb2hls (img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
def white_yellow_hls(img): 
    # get white and yellow colors in image only
    # white color mask
    
    white_mask_lower = np.uint8([  0, 200,   0])
    white_mask_upper = np.uint8([179, 255, 255])
    white_mask = cv2.inRange(img, white_mask_lower, white_mask_upper)
    # yellow color mask
    yellow_mask_lower = np.uint8([ 15,   0, 80])
    yellow_mask_upper = np.uint8([ 45, 255, 255])
    yellow_mask = cv2.inRange(img, yellow_mask_lower, yellow_mask_upper)

    mask = cv2.bitwise_or(white_mask, yellow_mask)
    return cv2.bitwise_and(img, img, mask = mask)
    

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=10):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    
    if lines is not None: # check point for images that do not have lines after hough transform        
        left_slope_arr = []
        left_slope_x = []
        left_slope_y = []
        right_slope_arr = []
        right_slope_x = []
        right_slope_y = []
        for line in lines:
            for x1,y1,x2,y2 in line:
                if (x2 - x1) == 0: # take out all vertical lines since slope --> inf
                    continue 
                slope = (y2 - y1) / (x2 - x1)
                if slope > 0 and (min(x1,x2) > (img.shape[1]>>1)): # x,y is backwards in plot domain, right slope is positive.
                    right_slope_arr.append(slope)
                    right_slope_x.extend([x1, x2])
                    right_slope_y.extend([y1, y2])
                elif slope < 0  and (max(x1,x2) < (img.shape[1]>>1)) :
                    left_slope_arr.append(slope)
                    left_slope_x.extend([x1, x2])
                    left_slope_y.extend([y1, y2])
        #print (right_slope_x, len(right_slope_x))
        
        
        right_slope_a, right_slope_b = np.polyfit(right_slope_x, right_slope_y,1)
        left_slope_a, left_slope_b = np.polyfit(left_slope_x, left_slope_y,1)
        
        
        #extrapolating 
        #x_min = min(right_slope_x_min, left_slope_x_min)
        #draw right slope line
        # x = (y-b)/a, set line boundaries to be the low side of the plot to close to the middle of the plot
        right_slope_x1 = int((img.shape[0] - right_slope_b ) / right_slope_a)
        right_slope_x2 = int(((img.shape[0] * 0.6 ) - right_slope_b ) / right_slope_a)
        cv2.line(img, (right_slope_x1, img.shape[0]), (right_slope_x2, int(img.shape[0] * 0.6)), color, thickness)
        #draw left slope line
        # x = (y-b)/a
        left_slope_x1 = int((img.shape[0] - left_slope_b ) / left_slope_a)
        left_slope_x2 = int(((img.shape[0] * 0.6 ) - left_slope_b ) / left_slope_a)
        cv2.line(img, (left_slope_x1, img.shape[0]), (left_slope_x2, int(img.shape[0] * 0.6)), color, thickness)

def draw_lines_no_averaging(img, lines, color=[255, 0, 0], thickness=2):
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1.0, λ=0.0):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

Processing test images

In [4]:
#parse each file in test image folder
for img in os.listdir("test_images/"): 
    image = mpimg.imread("test_images/"+img)
    line_image = np.copy(image)*1 # creating a blank to draw lines on
    test = np.copy(image)*1 # creating a blank to draw lines on
    image = rgb2hls(image)
    image = white_yellow_hls(image)
    
    create_dir("test_images_gray")
    gray = grayscale(image) 
    mpimg.imsave("test_images_gray/"+img, gray, cmap = cm.gray)
    
    # Define a kernel size and apply Gaussian smoothing

    kernel_arr = [13] #kernel_size = 13 used
    for kernel_size in kernel_arr:
    
        blur_gray = gaussian_blur(gray,kernel_size)
        low_threshold = 50
        high_threshold = 150
        edges = canny(blur_gray, low_threshold, high_threshold)

    # This time we are defining a four sided polygon to mask
    imshape = image.shape
    vertices = np.array([[(imshape[1]-30,imshape[0]),(80, imshape[0]), ((imshape[1]>>1)-60, (imshape[0]>>1)+60), ((imshape[1]>>1)+60, (imshape[0]>>1)+60)]], dtype=np.int32)
    masked_edges = region_of_interest(edges, vertices)

    # Define the Hough transform parameters
    # Make a blank the same size as our image to draw on
    rho = 1 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 15     # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 10 #minimum number of pixels making up a line
    max_line_gap = 40    # maximum gap in pixels between connectable line segments
    

    # Run Hough on edge detected image
    # Output is an array containing endpoints of detected line segments

    
    line_img = hough_lines(masked_edges, rho, theta, threshold, 
                                min_line_length, max_line_gap)
    
    

    line_image = weighted_img(line_img,line_image)
    create_dir("test_images_output")
    mpimg.imsave("test_images_output/"+img, line_image)

**Video Processing**

Video processing was fairly straightforward since by using moviepy.editor to convert video clip to discrete images and then adding lane lines on the images based on the steps above to find lane line in the images. 


Packages used for video processing

In [5]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [6]:
def process_image(image):

    imshape = image.shape
    vertices = np.array([[(imshape[1]-30,imshape[0]),(80, imshape[0]), ((imshape[1]>>1)-60, (imshape[0]>>1)+60), ((imshape[1]>>1)+60, (imshape[0]>>1)+60)]], dtype=np.int32)

    #take image input and convert to HLS
    img1 = rgb2hls(image)
    
    img1 = white_yellow_hls(img1)

    img1 = grayscale(img1)
    img1 = gaussian_blur(img1,kernel_size)
    img1 = canny(img1,low_threshold, high_threshold)
    img1 = region_of_interest(img1,vertices)
    line_img = hough_lines(img1, rho, theta, threshold, min_line_length, max_line_gap)

    line_img = weighted_img(line_img,image)
    
    return line_img

Processing video images:

In [7]:
create_dir("test_videos_output")

white_output = 'test_videos_output/solidWhiteRight.mp4'

clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidWhiteRight.mp4
[MoviePy] Writing video test_videos_output/solidWhiteRight.mp4


100%|███████████████████████████████████████▊| 221/222 [00:06<00:00, 31.81it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidWhiteRight.mp4 

Wall time: 7.84 s


Output video results

In [8]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [9]:
create_dir("test_videos_output")

white_output = 'test_videos_output/solidYellowLeft.mp4'

clip1 = VideoFileClip("test_videos/solidYellowLeft.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidYellowLeft.mp4
[MoviePy] Writing video test_videos_output/solidYellowLeft.mp4


100%|███████████████████████████████████████▉| 681/682 [00:23<00:00, 29.41it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidYellowLeft.mp4 

Wall time: 23.9 s


In [10]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [11]:
create_dir("test_videos_output")

white_output = 'test_videos_output/challenge.mp4'

clip1 = VideoFileClip("test_videos/challenge.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/challenge.mp4
[MoviePy] Writing video test_videos_output/challenge.mp4


100%|████████████████████████████████████████| 251/251 [00:18<00:00, 15.04it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/challenge.mp4 

Wall time: 20.3 s


In [12]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

There's a couple of improvements that can be done to further optimize the performance. For example, the lane lines in the. ideo are slightly shaky, by applying IIR filter can help stablize the behavior (note that if heavy filtering is applied then the lane lines may not react fast enough so that is one concern of using IIR filtering). Also, the curvature of the lane lines are not captured in this project.  